In [1]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_wind_data(lat, lon, start_date, end_date):
    """
    Fetches daily wind speed (WS10M) and wind direction (WD10M) data 
    from the NASA POWER API for a given point.

    :param lat: Latitude of the location.
    :param lon: Longitude of the location.
    :param start_date: Start date in YYYYMMDD format.
    :param end_date: End date in YYYYMMDD format.
    :return: Pandas DataFrame with columns 'date', 'wind_speed', and 'wind_direction'.
    """
    # Build API URL with both parameters: WS10M and WD10M
    url = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?"
        f"parameters=WS10M,WD10M&community=RE&longitude={lon}&latitude={lat}"
        f"&start={start_date}&end={end_date}&format=JSON"
    )

    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        parameters = data.get("properties", {}).get("parameter", {})
        if "WS10M" in parameters and "WD10M" in parameters:
            # Create separate DataFrames for wind speed and wind direction
            ws_dict = parameters["WS10M"]
            wd_dict = parameters["WD10M"]
            df_ws = pd.DataFrame(ws_dict.items(), columns=["date", "wind_speed"])
            df_wd = pd.DataFrame(wd_dict.items(), columns=["date", "wind_direction"])
            # Merge the two DataFrames on the date column
            df = pd.merge(df_ws, df_wd, on="date")
            # Convert the date column to datetime format
            df["date"] = pd.to_datetime(df["date"], format='%Y%m%d')
            return df
        else:
            print("⚠️ No wind speed or direction data found in the response.")
            return None
    else:
        print(f"❌ Error fetching data: {response.status_code}")
        print(response.text)
        return None

# Set location and date parameters for Lankien, South Sudan
latitude = 8.4281
longitude = 33.7833
start = "20150101"  # Fixed start date
# Set dynamic end date as 7 days before today (to ensure complete data)
end = (datetime.today() - timedelta(days=7)).strftime("%Y%m%d")

# Fetch wind data
wind_data = get_wind_data(latitude, longitude, start, end)

# Save data to CSV if valid data is returned
if wind_data is not None and not wind_data.empty:
    output_folder = "Wind_Data_Dataset"
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f"wind_Lankien_{end}.csv")
    wind_data.to_csv(output_path, index=False)
    print(f"✅ Data saved successfully to {output_path}")
    print(wind_data.head())
else:
    print("⚠️ No valid wind data found for the specified location and time range.")


✅ Data saved successfully to Wind_Data_Dataset/wind_Lankien_20250304.csv
        date  wind_speed  wind_direction
0 2015-01-01        1.59            41.8
1 2015-01-02        1.48            84.5
2 2015-01-03        1.61           344.8
3 2015-01-04        3.90           349.7
4 2015-01-05        3.72           337.0


In [4]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta

def get_wind_data(lat, lon, start_date, end_date):
    """
    Fetches daily wind speed and wind direction data at 10m and 50m
    from the NASA POWER API for a given latitude and longitude.

    :param lat: Latitude of the location.
    :param lon: Longitude of the location.
    :param start_date: Start date in YYYYMMDD format.
    :param end_date: End date in YYYYMMDD format.
    :return: Pandas DataFrame with wind speed and direction.
    """
    # API URL (removing WS100M and WD100M)
    url = (
        f"https://power.larc.nasa.gov/api/temporal/daily/point?"
        f"parameters=WS10M,WD10M,WS50M,WD50M"
        f"&community=RE&longitude={lon}&latitude={lat}"
        f"&start={start_date}&end={end_date}&format=JSON"
    )

    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        parameters = data.get("properties", {}).get("parameter", {})

        if all(param in parameters for param in ["WS10M", "WD10M", "WS50M", "WD50M"]):
            # Extract wind data for each height level
            df = pd.DataFrame({
                "date": parameters["WS10M"].keys(),
                "wind_speed_10m": parameters["WS10M"].values(),
                "wind_direction_10m": parameters["WD10M"].values(),
                "wind_speed_50m": parameters["WS50M"].values(),
                "wind_direction_50m": parameters["WD50M"].values()
            })
            # Convert date column to datetime
            df["date"] = pd.to_datetime(df["date"], format='%Y%m%d')
            return df
        else:
            print("⚠️ Some wind data parameters are missing in the response.")
            return None
    else:
        print(f"❌ Error fetching data: {response.status_code}")
        print(response.text)
        return None

# Set location and date parameters for Lankien, South Sudan
latitude = 8.4281
longitude = 33.7833
start = "20150101"  # Fixed start date
end = (datetime.today() - timedelta(days=7)).strftime("%Y%m%d")  # 7 days before today

# Fetch wind data
wind_data = get_wind_data(latitude, longitude, start, end)

# Save data to CSV if valid data is returned
if wind_data is not None and not wind_data.empty:
    output_folder = "Wind_Data_Dataset"
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f"wind_Lankien_{end}.csv")
    wind_data.to_csv(output_path, index=False)
    print(f"✅ Data saved successfully to {output_path}")
    print(wind_data.head())
else:
    print("⚠️ No valid wind data found for the specified location and time range.")


✅ Data saved successfully to Wind_Data_Dataset/wind_Lankien_20250304.csv
        date  wind_speed_10m  wind_direction_10m  wind_speed_50m  \
0 2015-01-01            1.59                41.8            2.65   
1 2015-01-02            1.48                84.5            2.30   
2 2015-01-03            1.61               344.8            2.57   
3 2015-01-04            3.90               349.7            6.15   
4 2015-01-05            3.72               337.0            5.77   

   wind_direction_50m  
0                29.1  
1                77.3  
2               348.6  
3               350.7  
4               337.8  
